#Init

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
# Imports here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import cv2

import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from torch.optim import lr_scheduler
from torch.utils.data import WeightedRandomSampler, DataLoader

In [ ]:
data_dir = 'PyTorch_Image_Classifier/flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
!pip install kaggle 
!pip install --upgrade --force-reinstall --no-deps kaggle

# Download Kaggle Cassava Dataset

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle competitions download -c cassava-leaf-disease-classification # api copied from kaggle

In [ ]:
!unzip cassava-leaf-disease-classification.zip

In [ ]:
df = pd.read_csv("train.csv")

# Test Train Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df["image_id"], df["label"], test_size=0.15, random_state=42)

#for training and validing
train_data, val_data, train_label, val_label = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

# Making Directory In Colab

In [ ]:
!mkdir "train_i"
!mkdir "val_i"
!mkdir "test_i" 

for label in range(5):
  if not os.path.exists("train_i/"+str(label)):
    os.makedirs("train_i/"+str(label))
  
  if not os.path.exists("val_i/"+str(label)):
    os.makedirs("val_i/"+str(label))
  
  if not os.path.exists("test_i/"+str(label)):
    os.makedirs("test_i/"+str(label))

import os
import shutil 
import os

for x,label in zip(train_data,train_label):
  #cmd = f"cp -a /train_images/{x}/. /train_1/"

  send_path = "/content/train_images/"+x
  print(send_path)
  #!mv send_path "/train_1"
  shutil.move(send_path, "/content/train_i/"+str(label)) 
  
for x,label in zip(val_data,val_label):
  #cmd = f"cp -a /train_images/{x}/. /train_1/"
  send_path = "/content/train_images/"+x
  print(send_path)
  #!mv send_path "/train_1"
  shutil.move(send_path, "/content/val_i/"+str(label)) 

for x,label in zip(X_test,y_test):
  #cmd = f"cp -a /train_images/{x}/. /train_1/"
  send_path = "/content/train_images/"+x
  print(send_path)
  #!mv send_path "/train_1"
  shutil.move(send_path, "/content/test_i/"+str(label))   

In [ ]:
from PIL import Image
image = Image.open('train_i/0/' + train_data[19689])
array = np.array(image)
array.shape
#plt.imshow(array)
#train_label[19689]

# DenseNet

In [ ]:
def get_WeightedSamplerDataLoader(training_dataset,batch_size):
  class_weights = {}
  sample_weights = [0] * len(training_dataset)

  for root, subdir, files in os.walk("/content/train_i"):
    if len(root.split("/")) == 4:
      class_weights[str(root.split("/")[3])] = round((10/len(files))*100,7)
  
  for idx, (data,label) in enumerate(training_dataset.imgs):
    weights = class_weights[str(label)]
    sample_weights[idx] = weights 

  sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights) , replacement=True)

  loader = DataLoader(training_dataset, batch_size = batch_size, sampler= sampler)

  return loader

In [ ]:
for root, subdir, files in os.walk("/content/train_i"):
  #print(len(root.split("/")))
  if len(root.split("/")) == 4:
    #print("Belongs to folder "+str(root.split("/")[3]))
    print(len(files))
    print((10/len(files))*100)

In [ ]:
training_transforms = transforms.Compose([transforms.RandomRotation(30),
                                          transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.RandomVerticalFlip(),
                                          transforms.ColorJitter(brightness=0.10, contrast=0.2, saturation=0.2, hue=0.00),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])

testing_transforms = transforms.Compose([transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406], # This is Mean
                                                              [0.229, 0.224, 0.225])]) # This Standard Deviation

# TODO: Load the datasets with ImageFolder
training_dataset = datasets.ImageFolder("train_i/", transform=training_transforms)
validation_dataset = datasets.ImageFolder("val_i/", transform=validation_transforms)
testing_dataset = datasets.ImageFolder("test_i/", transform=testing_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
#train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=True)
train_loader = get_WeightedSamplerDataLoader(training_dataset,64)
validate_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32)
test_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=32)

In [ ]:
# Build and train your network
# Transfer Learning
#model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)
model = torchvision.models.densenet121(pretrained=True)

In [ ]:
print(model)

In [ ]:
# Function for the validation pass
def validation(model, validateloader, criterion):
    
    val_loss = 0
    accuracy = 0
    
    for images, labels in iter(validateloader):

        images, labels = images.to('cuda'), labels.to('cuda')

        output = model.forward(images)
        val_loss += criterion(output, labels).item()

        probabilities = torch.exp(output)
        
        equality = (labels.data == probabilities.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return val_loss, accuracy

In [ ]:
# Freeze pretrained model parameters to avoid backpropogating through them
for parameter in model.parameters():
    parameter.requires_grad = False

model.classifier = nn.Linear(1024, 5)

#criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.AdamW(model.classifier.parameters(), lr=0.001,eps=1e-08, weight_decay=0.01)

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [ ]:
# Train the classifier

#from workspace_utils import active_session

def train_classifier():

    #with active_session():

      epochs = 15
      steps = 0
      print_every = 40

      model.to('cuda')

      for e in range(epochs):
      
          model.train()

          running_loss = 0
  
          for images, labels in iter(train_loader):
      
              steps += 1
      
              images, labels = images.to('cuda'), labels.to('cuda')
      
              optimizer.zero_grad()
      
              output = model.forward(images)
              loss = criterion(output, labels)
              loss.backward()
              optimizer.step()
      
              running_loss += loss.item()
      
              if steps % print_every == 0:
              
                  model.eval()
              
                  # Turn off gradients for validation, saves memory and computations
                  with torch.no_grad():
                      validation_loss, accuracy = validation(model, validate_loader, criterion)
          
                  print("Epoch: {}/{}.. ".format(e+1, epochs),
                        "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                        "Validation Loss: {:.3f}.. ".format(validation_loss/len(validate_loader)),
                        "Validation Accuracy: {:.3f}".format(accuracy/len(validate_loader)))
          
                  running_loss = 0
                  #exp_lr_scheduler.step()
                  model.train()
                  
train_classifier()

In [ ]:
def test_accuracy(model, test_loader):

    # Do validation on the test set
    model.eval()
    model.to('cuda')

    with torch.no_grad():
    
        accuracy = 0
    
        for images, labels in iter(test_loader):
    
            images, labels = images.to('cuda'), labels.to('cuda')
    
            output = model.forward(images)

            probabilities = torch.exp(output)
        
            equality = (labels.data == probabilities.max(dim=1)[1])
        
            accuracy += equality.type(torch.FloatTensor).mean()
        
        print("Test Accuracy: {}".format(accuracy/len(test_loader)))    
        
        
test_accuracy(model, test_loader)

In [ ]:
def save_checkpoint(model):

    model.class_to_idx = training_dataset.class_to_idx

    checkpoint = {'arch': "DenseNet121",
                  'class_to_idx': model.class_to_idx,
                  'model_state_dict': model.state_dict()
                 }

    torch.save(checkpoint, 'checkpoint_DenseNet_PreTrained_NoFreeze_AdamW_0.005.pth')
    
#save_checkpoint(model)

In [ ]:
from collections import OrderedDict

# Function that loads a checkpoint and rebuilds the model

def load_checkpoint(filepath):

    checkpoint = torch.load(filepath, map_location=torch.device('cpu'))
    
    if checkpoint['arch'] == 'DenseNet121':
        
        model = torchvision.models.densenet121(pretrained=True)
        
        for param in model.parameters():
            param.requires_grad = False
    else:
        print("Architecture not recognized.")
    
    #model.class_to_idx = checkpoint['class_to_idx']
    
#     model.classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(1024, 512)),
#                                     ('relu', nn.ReLU()),
#                                     ('drop', nn.Dropout(p=0.5)),
#                                     ('fc2', nn.Linear(512, 5)),
#                                     ('output', nn.LogSoftmax(dim=1))]))

    model.classifier = nn.Linear(1024, 5)
    
    model.load_state_dict(checkpoint["model_state_dict"])
    
    return model

model = load_checkpoint('checkpoint_DenseNet_PreTrained_NoFreeze_AdamW_0.005.pth')
print(model)

In [ ]:
from PIL import Image

testing_transforms = transforms.Compose([transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406], # This is Mean
                                                              [0.229, 0.224, 0.225])]) # This Standard Deviation

test_img = Image.open(r'C:\Users\rehma\Documents\Python Scripts\Kaggle\cassava-leaf-disease-classification\test_images\2216849948.jpg')
array = np.array(test_img)
plt.imshow(array)
tranform_test_image = testing_transforms(test_img)


In [ ]:
plt.imshow(tranform_test_image.permute(1, 2, 0))

In [ ]:
output = model.forward(tranform_test_image[None])
probabilities = torch.exp(output)
probabilities.max(dim=1)[1]

In [ ]:
tranform_test_image[None].shape

In [ ]:
probabilities

In [ ]:
df = pd.read_csv("cassava-leaf-disease-classification\sample_submission.csv")
df

In [ ]:
df["label"] = probabilities.max(dim=1)[1]

In [ ]:
df.to_csv('submission.csv', index=False)